___
# Ciência dos Dados - PROJETO 2

___
## Giovanni dos Santos

## Gustavo Tessitore

## Victor Niubó

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [2]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import os.path
import numpy as np
import math
from random import shuffle

In [3]:
emails = pd.read_excel("spamham2019.xlsx") #lê o excel
remover = [',','@','/',':',';','r$','\n','\(','\)','#','\?','-','"','\[','\]']

for i in remover:
    emails.Email = emails.Email.replace(i,'',regex=True)

In [4]:
dicio = emails.to_dict('records')
shuffle(dicio)
df2 = pd.DataFrame.from_dict(dicio)
df3 = df2.head(int(len(df2)*0.75))
df4 = df2.tail(int(len(df2)*0.25))

___
## 3. Classificador Naive-Bayes

In [5]:
spam = df3.Class

prob = spam.value_counts()

prob_ham = prob[0]/len(spam) 

prob_spam = prob[1]/len(spam) 

print("Probabilidade de ser relevante:",int(prob_ham * 100),'%')
print('Probabilidade de ser irrelevante',int(prob_spam * 100),'%')

Probabilidade de ser relevante: 86 %
Probabilidade de ser irrelevante 13 %


In [6]:
presunto = df3[df3.Class == 'ham'] #Separa os presuntos relevantes
relevantes = np.sum(presunto.Email).split()
quant_em_relev =  pd.Series(relevantes)
todos_relevantes = len(relevantes)
quant_em_relev.value_counts().head(3)

to     1152
you    1113
I       799
dtype: int64

In [7]:
queijo = df3[df3.Class == 'spam'] #Separa os queijos relevantes
irelevantes = np.sum(queijo.Email).split()
quant_em_irelev =  pd.Series(irelevantes)
todos_irelevantes = len(irelevantes)
quant_em_irelev.value_counts().head(3)

todas_as_palavras = irelevantes + relevantes

In [48]:
def P_spam_palavra(palavra):
    if palavra in irelevantes:
        return ((1 + quant_em_irelev.value_counts()[palavra])/(len(todas_as_palavras) + len(irelevantes)))
    else:
        return 1/(len(todas_as_palavras) + len(irelevantes))
    
def P_palavra_spam(palavra):
    x = (P_spam_palavra(palavra) * P_palavra(palavra)) / prob_spam
    return x

def P_palavra(palavra):
    quant_pal = (quant_em_irelev.value_counts()[palavra] + quant_em_relev.value_counts()[palavra])
    quant_tot = len(irelevantes)+len(relevantes)
    return (quant_pal/quant_tot)

def P_ham_palavra(palavra):
    if palavra in relevantes:
        return ((1 + quant_em_relev.value_counts()[palavra])/(len(todas_as_palavras) + len(relevantes)))
    else:
        return 1/(len(todas_as_palavras) + len(irelevantes))
    
def P_palavra_ham(palavra):
    x = (P_ham_palavra(palavra) * P_palavra(palavra)) / prob_ham
    return x

def P_spam_mensagem(mensagem):
    palavras = mensagem.split(' ')
    prob = 0
    contador = 0
    for palavra in palavras:
        if palavra in todas_as_palavras:
            if contador == 0:
                prob += P_spam_palavra(palavra)
            else:
                prob *= P_spam_palavra(palavra)
        else:
            pass
    return prob

def P_ham_mensagem(mensagem):
    palavras = mensagem.split(' ')
    prob = 0
    contador = 0
    for palavra in palavras:
        if palavra in todas_as_palavras:
            if contador == 0:
                prob += P_ham_palavra(palavra)
            else:
                prob *= P_ham_palavra(palavra)
        else:
            pass
    return prob

def Hamlet(mensagem):
    ham = P_ham_mensagem(mensagem)
    spam = P_spam_mensagem(mensagem)
    if ham > spam:
        return 'ham'
    else:
        return 'spam'

test = []
for i in range(len(df3.Email)):
    test.append(Hamlet(df3.Email[i]))

df3['NB'] = test

C:\Users\gugat\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
falso_neg = 0
falso_pos = 0
pos_ver = 0
neg_ver = 0
Spam = 0
Ham = 0

for e in range(500):
    if df3.Class[e] == 'spam':
        Spam += 1
        if df3.NB[e] == 'spam':
            pos_ver += 1
        
        elif df3.NB[e] != 'spam':
            falso_neg += 1
    
    elif df3.Class[e] == 'ham':
        Ham += 1
        if df3.NB[e] == 'ham' :
            neg_ver += 1
        
        elif df3.NB[e] != 'ham':
            falso_pos += 1
            
total = falso_neg + falso_pos + pos_ver + neg_ver            

print('Porcentagem de falsos positivos: {0}%'.format(falso_pos/ * 100))
print('Porcentagem de positivos verdadeiros: {0}%'.format(pos_ver/ * 100))
print('Porcentagem de falsos negativos: {0}%'.format(falso_neg/ * 100))
print('Porcentagem de negativos verdadeiros: {0}%'.format(neg_ver/ * 100))
print(total)

Porcentagem de falsos positivos: 0.4%
Porcentagem de positivos verdadeiros: 1.6%
Porcentagem de falsos negativos: 11.4%
Porcentagem de negativos verdadeiros: 86.6%
500


___
## 4. Qualidade do Classificador alterando a base de treinamento